In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df_match_details = pd.read_csv('../data/raw/match_details.csv')
raw_df_player_attr = pd.read_csv('../data/raw/player_attributes.csv')

In [3]:
raw_df_player_attr['overall_rating_last_5'] = raw_df_player_attr.groupby('player_api_id')['overall_rating'].transform(
    lambda x: x.shift().rolling(5).mean()
)

In [4]:
# raw_df_match_details.info()

In [5]:
# raw_df_player_attr.info()

In [6]:
# (raw_df_match_details
#  .assign(date=pd.to_datetime(raw_df_match_details['date']),
#          season=pd.to_datetime(raw_df_match_details['season'].astype('')),)
#  )

In [7]:
raw_df_match_details['date'] = pd.to_datetime(raw_df_match_details['date'])
raw_df_player_attr['date'] = pd.to_datetime(raw_df_player_attr['date'])
raw_df_match_details.sort_values(by='date', inplace=True)

In [8]:
from playerstats import player_stats

players_cols = ['{}_player_{}'.format(team, i) for team in ['home', 'away'] for i in range(1, 12)]

player_stats_dict_series = raw_df_match_details.apply(
    lambda row: player_stats.calculate_player_stat(
        match_row=row,
        df_matches=raw_df_match_details,
        df_player_attr=raw_df_player_attr,
        players=players_cols
    ),
    axis=1
)

new_player_stats_df = pd.json_normalize(player_stats_dict_series)

df = pd.merge(raw_df_match_details, new_player_stats_df, how='left', on='match_api_id')
df.drop(players_cols, axis=1, inplace=True)
matching_columns = [col for col in df.columns if 'player_rating' in col]
df.dropna(subset=matching_columns, inplace=True)